In [2]:
import torch 
import torch.nn as nn

In [27]:
class BasicBlock(nn.Module):
    def __init__(self,in_channels, out_channels, mid_channels):
        super(BasicBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding =1)
        self.conv2 = nn.Conv2d(mid_channels,out_channels ,kernel_size=3, padding =1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  #stride-> Kernel_size자동
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        return x

In [28]:
class VGG(nn.Module):
    def __init__(self,num_classes):
        super().__init__()  #nn.module 싹가져와
        
        
        #Conv Blocks 
        self.block1 = BasicBlock(in_channels=3, out_channels=32, mid_channels=16)   #B C 16 16
        self.block2 = BasicBlock(in_channels=32, out_channels=128, mid_channels=64)  # B C 8 8
        self.block3 = BasicBlock(in_channels=128, out_channels=256, mid_channels=128)  #  B C 4 4
    
        #Classifier
        self.fc1 = nn.Linear(4096,2048) # spatial size = (4*4) depth=256  4*4*256
        self.fc2 = nn.Linear(2048,256) #
        self.fc3 = nn.Linear(256,num_classes)
        
        self.relu = nn.ReLU()
        
        
    def forward(self,x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = torch.flatten(x, start_dim=1)
        
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        return x
        
    

In [29]:
from torch.utils.data.dataloader import DataLoader
from torchvision.transforms import Compose, RandomCrop, RandomHorizontalFlip, ToTensor, Normalize
from torch.optim.adam import Adam

transforms = Compose([
    RandomCrop((32,32),padding=4),  #PIL?네 맞습니다. CIFAR10이 32*32 인데 32*32 Cropping 하면 무슨의미가
    RandomHorizontalFlip(p=0.5), #PIL? 네
    ToTensor(), #어차피 Tensor인데 이건 또 왜하는거 텐서로, 이후 Normalize
    Normalize(mean=(0.5,0.5,0.5), std=(0.24,0.24,0.24)) #Normalize는 Tensor의 입력을 받는다.
])

from torchvision.datasets import CIFAR10

trainset = CIFAR10(root='./', train=True, download = True , transform = transforms) # PIL image
testset = CIFAR10(root='./', train=True, download = True, transform = transforms)


trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = VGG(num_classes=10)
model.to(device)







Files already downloaded and verified
Files already downloaded and verified


VGG(
  (block1): BasicBlock(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): BasicBlock(
    (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): BasicBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=4096, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=256,

In [32]:
#set config 
lr = 1e-5
optim = Adam(model.parameters(), lr=lr)
epoch = 20
criterion = nn.CrossEntropyLoss()

for epoch in range(epoch):
    for data,label in trainloader:
        optim.zero_grad()
        preds = model(data.to(device))
        
        loss = criterion(preds,label.to(device))
        loss.backward()
        optim.step()
        
    if epoch==0 or epoch%10==9:
        print(f'epoch{epoch+1} loss:{loss.item()}')
        

#모델 저장
torch.save(model.state_dict(), "CIFAR.pth")  # 모델저장은 근데 eval까지 돌리고 best parameter를 저장하던게 아닌가요
        
        
    
    

epoch1 loss:0.15882045030593872
epoch10 loss:0.4716978073120117
epoch20 loss:0.6751802563667297


In [39]:
model.load_state_dict(torch.load("CIFAR.pth",map_location = device)) # load_state_dict 대상을 device에 불러오는 option제공

num_corr = 0

with torch.no_grad():
    for data,label in testloader:
        output = model(data.to(device))
        #print(output.shape)
        preds = output.data.max(1)[1]  #output의 형태를 ? -> (32,10) , Batch, num_class == 마지막classifier output
        corr = preds.eq(label.to(device).data).sum().item()
        num_corr += corr
        
    print(f"Accuracy:{num_corr/len(testset)}")

Accuracy:0.86012
